In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('preprocessed_data.txt', header=None, names=['tokens'])

In [3]:
data.head()

,tokens
0,? (film) also written tanda tanya meaning ques...
1,0.999... mathematics 0999 also written 09 09 0...
2,1 − 2 + 3 − 4 + ⋯ mathematics 1 2 3 4 infinite...
3,1 Line (Sound Transit) 1 line formerly central...
4,1st Cavalry Division (Kingdom of Yugoslavia) 1...


In [4]:
tokens = data["tokens"]

In [7]:
# Import CountVectorizer from sklearn
from sklearn.feature_extraction.text import CountVectorizer

# Insantiate CountVectorizer
vectorizer = CountVectorizer()

tokenized_data = tokens

vectorizer.fit(tokenized_data)

CountVectorizer()

In [8]:
document_term_matrix = vectorizer.transform(tokenized_data)

In [9]:
# Print the document-term matrix as a dense array
print(document_term_matrix.toarray())

# Get the feature names
print(vectorizer.get_feature_names_out())

[[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['00' '000' '0000' ... 'đảng' 'đức' 'ōkami']


In [14]:
from sklearn.decomposition import NMF

# Define the number of components
n_components = 10

# Initialize NMF with specified number of components
nmf = NMF(n_components=n_components, init='nndsvd', random_state=0)
nmf_features = nmf.fit_transform(document_term_matrix)

# For loop to iterate through topics
for topic_idx in range(n_components):
    top_words = [vectorizer.get_feature_names_out()[i] for i in nmf.components_[topic_idx].argsort()[-10:]][::-1]
    print(f"Topic {topic_idx+1}: {top_words}")

Topic 1: ['state', 'would', 'new', 'president', 'government', 'party', 'american', 'united', 'war', 'election']
Topic 2: ['game', 'player', 'team', 'character', 'series', 'released', 'one', 'also', 'sonic', 'version']
Topic 3: ['film', 'character', 'role', 'time', 'one', 'scene', 'said', 'also', 'best', 'series']
Topic 4: ['specie', 'also', 'may', 'found', 'one', 'bird', 'used', 'known', 'year', 'two']
Topic 5: ['ship', 'fleet', 'gun', 'two', 'british', 'navy', 'battleship', 'sea', 'naval', 'squadron']
Topic 6: ['first', 'season', 'team', 'match', 'league', 'club', 'second', 'two', 'player', 'run']
Topic 7: ['album', 'song', 'music', 'band', 'released', 'single', 'first', 'record', 'one', 'number']
Topic 8: ['city', 'river', 'park', 'area', 'state', 'road', 'highway', 'north', 'building', 'new']
Topic 9: ['king', 'work', 'henry', 'year', 'first', 'one', 'also', 'time', 'england', 'later']
Topic 10: ['army', 'force', 'division', 'war', 'german', 'attack', 'battle', 'japanese', 'troop', 